In [ ]:
# Imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# XGBoost for tabular baseline
from xgboost import XGBRegressor

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

# Add src to path
sys.path.append('../src')
from data_fetcher import SatelliteImageFetcher

# Settings
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

from xgboost import XGBRegressor

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

import warnings
warnings.filterwarnings('ignore')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

In [ ]:
# Configuration
CONFIG = {
    'target_col': 'price',
    'lat_col': 'lat',
    'lon_col': 'long',
    'image_size': 224,
    'seed': 42,
    'test_size': 0.2,
    'batch_size': 32,
    'epochs': 15,
    'lr': 5e-4,
    'use_log_target': True,
}

# Paths - UPDATE THESE
DATA_DIR = '../data'
IMAGE_DIR = '../data/mapbox_images'
OUTPUT_DIR = '../outputs'
TRAIN_PATH = f'{DATA_DIR}/train.xlsx'
TEST_PATH = f'{DATA_DIR}/test.xlsx'

np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])

## 1. Load & Preprocess Data

In [ ]:
train_df = pd.read_excel(TRAIN_PATH, engine='openpyxl')
test_df = pd.read_excel(TEST_PATH, engine='openpyxl')

train_df.columns = [c.strip() for c in train_df.columns]
test_df.columns = [c.strip() for c in test_df.columns]

print(f"Train: {train_df.shape}, Test: {test_df.shape}")

In [ ]:
exclude_cols = {CONFIG['target_col'], 'date', 'id', CONFIG['lat_col'], CONFIG['lon_col']}
feature_cols = [c for c in train_df.columns if c not in exclude_cols and train_df[c].dtype in ['int64', 'float64']]
print(f"Features ({len(feature_cols)}): {feature_cols}")

In [ ]:
train_data, val_data = train_test_split(train_df, test_size=CONFIG['test_size'], random_state=CONFIG['seed'])
train_image_ids = train_data.index.tolist()
val_image_ids = val_data.index.tolist()

print(f"Training: {len(train_data)}, Validation: {len(val_data)}")

In [ ]:
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

X_train = preprocessor.fit_transform(train_data[feature_cols])
X_val = preprocessor.transform(val_data[feature_cols])
X_test = preprocessor.transform(test_df[feature_cols])

if CONFIG['use_log_target']:
    y_train = np.log1p(train_data[CONFIG['target_col']].values)
    y_val = np.log1p(val_data[CONFIG['target_col']].values)
    y_train_original = train_data[CONFIG['target_col']].values
    y_val_original = val_data[CONFIG['target_col']].values
else:
    y_train = train_data[CONFIG['target_col']].values
    y_val = val_data[CONFIG['target_col']].values
    y_train_original = y_train
    y_val_original = y_val

print(f"X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")

## 2. Evaluation Function

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\n{model_name}: RMSE=${rmse:,.2f}, R²={r2:.4f}")
    return {'model': model_name, 'RMSE': rmse, 'R2': r2}

## 3. XGBoost Model

In [ ]:
xgb_model = XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=CONFIG['seed'],
    early_stopping_rounds=50,
    eval_metric='rmse',
    tree_method='hist',
    device='cuda' if DEVICE == 'cuda' else 'cpu'
)

xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

In [ ]:
xgb_pred_raw = xgb_model.predict(X_val)
if CONFIG['use_log_target']:
    xgb_pred = np.expm1(xgb_pred_raw)
    xgb_results = evaluate_model(y_val_original, xgb_pred, 'XGBoost')
else:
    xgb_results = evaluate_model(y_val, xgb_pred_raw, 'XGBoost')

## 4. Hybrid CNN+Tabular Model

In [ ]:
class PropertyDataset(Dataset):
    def __init__(self, X_tabular, y=None, image_ids=None, train_mode=True):
        self.X_tab = X_tabular.astype(np.float32)
        self.y = y.astype(np.float32) if y is not None else None
        self.image_ids = image_ids
        
        if train_mode:
            self.transform = T.Compose([
                T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
                T.RandomHorizontalFlip(0.5),
                T.RandomVerticalFlip(0.5),
                T.RandomRotation(15),
                T.ColorJitter(brightness=0.2, contrast=0.2),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.transform = T.Compose([
                T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        
        self.blank = Image.new('RGB', (CONFIG['image_size'], CONFIG['image_size']), (128, 128, 128))
    
    def __len__(self):
        return len(self.X_tab)
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx] if self.image_ids is not None else idx
        img_path = os.path.join(IMAGE_DIR, f'img_{int(img_id)}.png')
        
        try:
            img = Image.open(img_path).convert('RGB') if os.path.exists(img_path) else self.blank
        except:
            img = self.blank
        
        img_tensor = self.transform(img)
        tab_tensor = torch.from_numpy(self.X_tab[idx])
        
        if self.y is None:
            return img_tensor, tab_tensor
        return img_tensor, tab_tensor, torch.tensor(self.y[idx])

In [ ]:
class HybridModel(nn.Module):
    def __init__(self, tabular_dim):
        super().__init__()
        
        backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        backbone.fc = nn.Identity()
        self.cnn = backbone
        for param in self.cnn.parameters():
            param.requires_grad = False
        
        self.cnn_processor = nn.Sequential(
            nn.Linear(512, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, 64), nn.ReLU()
        )
        
        self.tabular = nn.Sequential(
            nn.Linear(tabular_dim, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU()
        )
        
        self.head = nn.Sequential(
            nn.Linear(128, 64), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, 1)
        )
    
    def forward(self, img, tab):
        with torch.no_grad():
            img_feat = self.cnn(img)
        img_feat = self.cnn_processor(img_feat)
        tab_feat = self.tabular(tab)
        return self.head(torch.cat([img_feat, tab_feat], dim=1)).squeeze(1)

In [ ]:
# Prepare data loaders for Hybrid model
y_train_hybrid = y_train_original.astype(np.float32)
y_val_hybrid = y_val_original.astype(np.float32)

train_dataset = PropertyDataset(X_train, y_train_hybrid, train_image_ids, True)
val_dataset = PropertyDataset(X_val, y_val_hybrid, val_image_ids, False)

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

In [ ]:
# Train Hybrid Model
model = HybridModel(X_train.shape[1]).to(DEVICE)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3, weight_decay=1e-4)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

best_rmse = float('inf')
best_state = None

for epoch in range(1, CONFIG['epochs'] + 1):
    model.train()
    for img, tab, y in tqdm(train_loader, desc=f'Epoch {epoch}', leave=False):
        img, tab, y = img.to(DEVICE), tab.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(img, tab), y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    
    model.eval()
    preds = []
    with torch.no_grad():
        for img, tab, y in val_loader:
            preds.extend(model(img.to(DEVICE), tab.to(DEVICE)).cpu().numpy())
    
    val_rmse = np.sqrt(mean_squared_error(y_val_hybrid, preds))
    scheduler.step(val_rmse)
    
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        print(f"Epoch {epoch}: Val RMSE=${val_rmse:,.0f} ✓")
    else:
        print(f"Epoch {epoch}: Val RMSE=${val_rmse:,.0f}")

print(f"\nBest Val RMSE: ${best_rmse:,.0f}")

In [ ]:
# Evaluate Hybrid Model
model.load_state_dict(best_state)
model.eval()
preds = []
with torch.no_grad():
    for img, tab, _ in val_loader:
        preds.extend(model(img.to(DEVICE), tab.to(DEVICE)).cpu().numpy())

hybrid_results = evaluate_model(y_val_original, np.array(preds), 'Hybrid')

## 5. Model Comparison

In [ ]:
results_df = pd.DataFrame([xgb_results, hybrid_results])
print("\n" + "="*50)
print("MODEL COMPARISON")
print("="*50)
display(results_df.round(2))

In [ ]:
# Visualization
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

ax[0].bar(results_df['model'], results_df['RMSE'], color=['steelblue', 'coral'])
ax[0].set_title('RMSE by Model')
ax[0].set_ylabel('RMSE ($)')

ax[1].bar(results_df['model'], results_df['R2'], color=['steelblue', 'coral'])
ax[1].set_title('R² by Model')
ax[1].set_ylabel('R²')

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/model_comparison.png', dpi=150)
plt.show()

## 6. Save Models

In [ ]:
import joblib

# Save XGBoost model
joblib.dump(xgb_model, f'{OUTPUT_DIR}/xgb_model.joblib')
print(f"Saved XGBoost model to {OUTPUT_DIR}/xgb_model.joblib")

# Save Hybrid model
torch.save(best_state, f'{OUTPUT_DIR}/hybrid_model.pth')
print(f"Saved Hybrid model to {OUTPUT_DIR}/hybrid_model.pth")

# Save preprocessor
joblib.dump(preprocessor, f'{OUTPUT_DIR}/preprocessor.joblib')
print(f"Saved preprocessor to {OUTPUT_DIR}/preprocessor.joblib")

## 7. Generate Predictions

In [ ]:
# Generate predictions using XGBoost model
xgb_test_preds_raw = xgb_model.predict(X_test)
test_preds = np.expm1(xgb_test_preds_raw) if CONFIG['use_log_target'] else xgb_test_preds_raw

submission = pd.DataFrame({
    'id': test_df['id'] if 'id' in test_df.columns else range(len(test_preds)),
    'predicted_price': test_preds
})

submission.to_csv(f'{OUTPUT_DIR}/predictions.csv', index=False)
print(f"Saved {len(test_preds)} predictions to {OUTPUT_DIR}/predictions.csv")
submission.head(10)

In [ ]:
print("\n✅ Training complete!")
print(f"XGBoost RMSE: ${xgb_results['RMSE']:,.0f}")
print(f"Hybrid RMSE: ${hybrid_results['RMSE']:,.0f}")

## 1. Load and Prepare Data

In [ ]:
# Configuration
CONFIG = {
    'train_path': '../data/train.xlsx',
    'test_path': '../data/test.xlsx',
    'target_col': 'price',
    'lat_col': 'lat',
    'lon_col': 'long',
    'image_cache': '../image_cache',
    'image_size': 224,
    'seed': 42,
    'test_size': 0.2,
    'batch_size': 32,
    'epochs': 15,
    'lr': 1e-4,
}

np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])

In [ ]:
# Load data
train_df = pd.read_excel(CONFIG['train_path'], engine='openpyxl')
test_df = pd.read_excel(CONFIG['test_path'], engine='openpyxl')

train_df.columns = [c.strip() for c in train_df.columns]
test_df.columns = [c.strip() for c in test_df.columns]

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

In [ ]:
# Define feature columns (exclude id, date, target, coordinates for tabular model)
exclude_cols = {CONFIG['target_col'], 'date', 'id', CONFIG['lat_col'], CONFIG['lon_col']}
feature_cols = [c for c in train_df.columns if c not in exclude_cols and train_df[c].dtype in ['int64', 'float64']]
print(f"Feature columns ({len(feature_cols)}): {feature_cols}")

In [ ]:
# Train/validation split
train_data, val_data = train_test_split(train_df, test_size=CONFIG['test_size'], random_state=CONFIG['seed'])
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

In [ ]:
# Preprocessing pipeline
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

X_train = preprocessor.fit_transform(train_data[feature_cols])
X_val = preprocessor.transform(val_data[feature_cols])
X_test = preprocessor.transform(test_df[feature_cols])

y_train = train_data[CONFIG['target_col']].values
y_val = val_data[CONFIG['target_col']].values

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

## 2. Baseline Model: Tabular-Only (XGBoost)

In [ ]:
# Train XGBoost baseline
xgb_model = XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=CONFIG['seed'],
    early_stopping_rounds=50,
    eval_metric='rmse'
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=50
)

In [ ]:
# Evaluate XGBoost
def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n📊 {model_name} Results:")
    print(f"   RMSE:  ${rmse:,.2f}")
    print(f"   MAE:   ${mae:,.2f}")
    print(f"   R²:    {r2:.4f}")
    print(f"   MAPE:  {mape:.2f}%")
    
    return {'model': model_name, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'MAPE': mape}

xgb_val_pred = xgb_model.predict(X_val)
xgb_results = evaluate_model(y_val, xgb_val_pred, 'XGBoost (Tabular Only)')

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
importance_df.head(15).plot(kind='barh', x='feature', y='importance', ax=ax, color='steelblue')
ax.set_xlabel('Feature Importance')
ax.set_title('XGBoost Feature Importance (Top 15)')
ax.invert_yaxis()
plt.tight_layout()
plt.savefig('../outputs/xgb_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Deep Learning Dataset and Models

In [ ]:
# Image directory with pre-downloaded images
IMAGE_DIR = '/Users/ruthwik/Downloads/mapbox_images'

# Auto-detect image naming convention
print("\n\ud83d\udd0d Detecting image naming convention...")
if 'id' in train_df.columns:
    test_id = train_df.iloc[0]['id']
    test_by_id = os.path.exists(os.path.join(IMAGE_DIR, f'img_{int(test_id)}.png'))
else:
    test_by_id = False
test_by_idx = os.path.exists(os.path.join(IMAGE_DIR, 'img_0.png'))

USE_ID_FOR_IMAGES = test_by_id
print(f"\u2705 Images by ID: {test_by_id}, by index: {test_by_idx}")
print(f"\u27a1\ufe0f Using {'ID column' if USE_ID_FOR_IMAGES else 'row index'} for image lookup")

class PropertyDataset(Dataset):
    """Dataset for hybrid tabular + image model - FIXED for ID-based image lookup."""
    
    def __init__(self, df, X_tabular, y=None, config=CONFIG, train_mode=True, image_ids=None):
        self.df = df.reset_index(drop=True)
        self.X_tab = X_tabular.astype(np.float32)
        self.y = y.astype(np.float32) if y is not None else None
        self.config = config
        # Store image IDs for correct lookup (img_{id}.png)
        self.image_ids = image_ids if image_ids is not None else list(range(len(df)))
        
        if train_mode:
            self.transform = T.Compose([
                T.Resize((config['image_size'], config['image_size'])),
                T.RandomHorizontalFlip(0.5),
                T.RandomVerticalFlip(0.1),
                T.ColorJitter(brightness=0.2, contrast=0.2),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.transform = T.Compose([
                T.Resize((config['image_size'], config['image_size'])),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        
        self.blank = Image.new('RGB', (config['image_size'], config['image_size']), (128, 128, 128))
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Get image by ID (not row index!)
        img_id = self.image_ids[idx]
        img_path = os.path.join(IMAGE_DIR, f'img_{int(img_id)}.png')
        
        img = self.blank
        if os.path.exists(img_path):
            try:
                img = Image.open(img_path).convert('RGB')
            except:

                pass        return img_tensor, tab_tensor, torch.tensor(self.y[idx])

                    return img_tensor, tab_tensor

        img_tensor = self.transform(img)        if self.y is None:

        tab_tensor = torch.from_numpy(self.X_tab[idx])        

In [ ]:
# CNN-Only Model (Image features only)
class CNNOnlyModel(nn.Module):
    """CNN model using only satellite images."""
    
    def __init__(self):
        super().__init__()
        backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        num_features = backbone.fc.in_features
        backbone.fc = nn.Identity()
        self.backbone = backbone
        
        self.head = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )
    
    def forward(self, img, tab=None):
        features = self.backbone(img)
        return self.head(features).squeeze(1)


# Hybrid Model (Tabular + Image fusion)
class HybridModel(nn.Module):
    """Hybrid model combining tabular and image features."""
    
    def __init__(self, tabular_dim):
        super().__init__()
        
        # CNN branch
        backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        cnn_features = backbone.fc.in_features
        backbone.fc = nn.Identity()
        self.cnn = backbone
        
        # Tabular branch
        self.tabular = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
        )
        
        # Fusion head
        self.head = nn.Sequential(
            nn.Linear(cnn_features + 64, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, img, tab):
        img_feat = self.cnn(img)
        tab_feat = self.tabular(tab)
        combined = torch.cat([img_feat, tab_feat], dim=1)
        return self.head(combined).squeeze(1)

In [ ]:
# Training functions
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for img, tab, y in tqdm(loader, desc='Training', leave=False):
        img, tab, y = img.to(device), tab.to(device), y.to(device)
        
        optimizer.zero_grad()
        pred = model(img, tab)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * len(y)
    
    return total_loss / len(loader.dataset)


@torch.no_grad()
def validate(model, loader, device):
    model.eval()
    preds, targets = [], []
    
    for img, tab, y in loader:
        img, tab = img.to(device), tab.to(device)
        pred = model(img, tab)
        preds.extend(pred.cpu().numpy())
        targets.extend(y.numpy())
    
    preds = np.array(preds)
    targets = np.array(targets)
    rmse = np.sqrt(mean_squared_error(targets, preds))
    return rmse, preds, targets


@torch.no_grad()
def predict(model, loader, device):
    model.eval()
    preds = []
    
    for img, tab in loader:
        img, tab = img.to(device), tab.to(device)
        pred = model(img, tab)
        preds.extend(pred.cpu().numpy())
    
    return np.array(preds)

## 4. Train Hybrid Model (Tabular + Satellite Images)

In [ ]:
# Create datasets and loaders
# Pass IMAGE IDs (from 'id' column) for correct image lookup (img_{id}.png)
if USE_ID_FOR_IMAGES and 'id' in train_df.columns:
    train_image_ids = train_data['id'].tolist()
    val_image_ids = val_data['id'].tolist()
    test_image_ids = test_df['id'].tolist() if 'id' in test_df.columns else list(range(len(test_df)))
    print("Using 'id' column for image lookup")
else:
    train_image_ids = train_data.index.tolist()
    val_image_ids = val_data.index.tolist()
    test_image_ids = list(range(len(test_df)))
    print("Using row index for image lookup")

print(f"Sample train image IDs: {train_image_ids[:5]}")

train_dataset = PropertyDataset(train_data, X_train, y_train, train_mode=True, image_ids=train_image_ids)
val_dataset = PropertyDataset(val_data, X_val, y_val, train_mode=False, image_ids=val_image_ids)
test_dataset = PropertyDataset(test_df, X_test, y=None, train_mode=False, original_indices=test_indices)

print(f"\\nImages loaded from: {IMAGE_DIR}")

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=0)print(f"Test batches: {len(test_loader)}")

val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=0)print(f"Val batches: {len(val_loader)}")

test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=0)print(f"Train batches: {len(train_loader)}")


In [ ]:
# Initialize hybrid model
hybrid_model = HybridModel(tabular_dim=X_train.shape[1]).to(DEVICE)
optimizer = torch.optim.AdamW(hybrid_model.parameters(), lr=CONFIG['lr'], weight_decay=1e-4)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

print(f"Model parameters: {sum(p.numel() for p in hybrid_model.parameters()):,}")

In [ ]:
# Training loop
best_rmse = float('inf')
best_state = None
history = {'train_loss': [], 'val_rmse': []}

for epoch in range(1, CONFIG['epochs'] + 1):
    train_loss = train_epoch(hybrid_model, train_loader, optimizer, criterion, DEVICE)
    val_rmse, val_preds, val_targets = validate(hybrid_model, val_loader, DEVICE)
    
    history['train_loss'].append(train_loss)
    history['val_rmse'].append(val_rmse)
    
    scheduler.step(val_rmse)
    
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_state = {k: v.cpu().clone() for k, v in hybrid_model.state_dict().items()}
        marker = '✓ (best)'
    else:
        marker = ''
    
    print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val RMSE: ${val_rmse:,.2f} {marker}")

print(f"\n🏆 Best Validation RMSE: ${best_rmse:,.2f}")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Training Loss')
axes[0].set_title('Training Loss')

axes[1].plot(history['val_rmse'], marker='o', color='orange')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Validation RMSE ($)')
axes[1].set_title('Validation RMSE')

plt.tight_layout()
plt.savefig('../outputs/training_history.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Load best model and evaluate
hybrid_model.load_state_dict(best_state)
_, hybrid_val_preds, _ = validate(hybrid_model, val_loader, DEVICE)
hybrid_results = evaluate_model(y_val, hybrid_val_preds, 'Hybrid Model (Tabular + Satellite)')

## 5. Model Comparison

In [ ]:
# Compare all models
results_df = pd.DataFrame([xgb_results, hybrid_results])
print("\n" + "=" * 60)
print("📊 MODEL COMPARISON")
print("=" * 60)
display(results_df.round(2))

In [ ]:
# Visualization: Model comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# RMSE comparison
axes[0].bar(results_df['model'], results_df['RMSE'], color=['steelblue', 'coral'])
axes[0].set_ylabel('RMSE ($)')
axes[0].set_title('RMSE Comparison (Lower is Better)')
axes[0].tick_params(axis='x', rotation=15)

# R² comparison
axes[1].bar(results_df['model'], results_df['R2'], color=['steelblue', 'coral'])
axes[1].set_ylabel('R² Score')
axes[1].set_title('R² Comparison (Higher is Better)')
axes[1].tick_params(axis='x', rotation=15)

# MAPE comparison
axes[2].bar(results_df['model'], results_df['MAPE'], color=['steelblue', 'coral'])
axes[2].set_ylabel('MAPE (%)')
axes[2].set_title('MAPE Comparison (Lower is Better)')
axes[2].tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.savefig('../outputs/model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Prediction vs Actual scatter plots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# XGBoost
axes[0].scatter(y_val, xgb_val_pred, alpha=0.5, s=10)
axes[0].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Price ($)')
axes[0].set_ylabel('Predicted Price ($)')
axes[0].set_title('XGBoost: Predicted vs Actual')

# Hybrid
axes[1].scatter(y_val, hybrid_val_preds, alpha=0.5, s=10, color='coral')
axes[1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Price ($)')
axes[1].set_ylabel('Predicted Price ($)')
axes[1].set_title('Hybrid Model: Predicted vs Actual')

plt.tight_layout()
plt.savefig('../outputs/predictions_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Grad-CAM Explainability

In [ ]:
import cv2

class GradCAM:
    """Grad-CAM implementation for CNN visualization."""
    
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Register hooks
        target_layer.register_forward_hook(self._forward_hook)
        target_layer.register_full_backward_hook(self._backward_hook)
    
    def _forward_hook(self, module, input, output):
        self.activations = output.detach()
    
    def _backward_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
    
    def generate(self, img_tensor, tab_tensor):
        """Generate Grad-CAM heatmap."""
        self.model.eval()
        
        # Forward pass
        output = self.model(img_tensor, tab_tensor)
        
        # Backward pass
        self.model.zero_grad()
        output.backward()
        
        # Compute weights
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        
        # Compute CAM
        cam = (weights * self.activations).sum(dim=1, keepdim=True)
        cam = torch.relu(cam)
        cam = cam.squeeze().cpu().numpy()
        
        # Normalize
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        
        return cam


def visualize_gradcam(model, img_path, tab_data, config, save_path=None):
    """Visualize Grad-CAM for a single property."""
    # Get target layer (last conv layer of ResNet)
    target_layer = model.cnn.layer4[-1].conv2
    gradcam = GradCAM(model, target_layer)
    
    # Load and preprocess image
    img = Image.open(img_path).convert('RGB')
    original_img = np.array(img.resize((config['image_size'], config['image_size'])))
    
    transform = T.Compose([
        T.Resize((config['image_size'], config['image_size'])),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    img_tensor = transform(img).unsqueeze(0).to(DEVICE)
    img_tensor.requires_grad_(True)
    
    tab_tensor = torch.from_numpy(tab_data.astype(np.float32)).unsqueeze(0).to(DEVICE)
    
    # Generate CAM
    cam = gradcam.generate(img_tensor, tab_tensor)
    
    # Resize CAM to image size
    cam_resized = cv2.resize(cam, (config['image_size'], config['image_size']))
    
    # Create heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    # Overlay
    overlay = np.uint8(0.4 * heatmap + 0.6 * original_img)
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(original_img)
    axes[0].set_title('Original Satellite Image')
    axes[0].axis('off')
    
    axes[1].imshow(cam_resized, cmap='jet')
    axes[1].set_title('Grad-CAM Heatmap')
    axes[1].axis('off')
    
    axes[2].imshow(overlay)
    axes[2].set_title('Grad-CAM Overlay\n(Areas influencing price prediction)')
    axes[2].axis('off')
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    return cam_resized

In [ ]:
# Visualize Grad-CAM for sample properties
sample_indices = val_data.sample(n=3, random_state=42).index.tolist()

for i, idx in enumerate(sample_indices):
    row = train_df.loc[idx]
    lat, lon = row[CONFIG['lat_col']], row[CONFIG['lon_col']]
    img_path = fetcher._get_cache_path(lat, lon)
    
    if os.path.exists(img_path):
        # Get tabular data for this sample
        tab_idx = val_data.index.get_loc(idx) if idx in val_data.index else 0
        tab_data = X_val[tab_idx] if tab_idx < len(X_val) else X_val[0]
        
        print(f"\n🏠 Property {i+1}: Price = ${row['price']:,.0f}")
        visualize_gradcam(
            hybrid_model, 
            img_path, 
            tab_data, 
            CONFIG,
            save_path=f'../outputs/gradcam_sample_{i+1}.png'
        )
    else:
        print(f"Image not found for sample {i+1}")

## 7. Generate Test Predictions

In [ ]:
# Generate final predictions using the hybrid model
hybrid_model.load_state_dict(best_state)
test_predictions = predict(hybrid_model, test_loader, DEVICE)

# Create submission dataframe with id and predicted_price
submission_df = pd.DataFrame({
    'id': test_df['id'] if 'id' in test_df.columns else range(len(test_predictions)),
    'predicted_price': test_predictions
})

# Save predictions
submission_df.to_csv('../outputs/predictions.csv', index=False)
print(f"✅ Predictions saved to ../outputs/predictions.csv")
print(f"   Shape: {submission_df.shape}")
display(submission_df.head())

In [ ]:
# Prediction statistics
print("\n📊 Prediction Statistics:")
print(f"   Mean:   ${test_predictions.mean():,.2f}")
print(f"   Median: ${np.median(test_predictions):,.2f}")
print(f"   Min:    ${test_predictions.min():,.2f}")
print(f"   Max:    ${test_predictions.max():,.2f}")
print(f"   Std:    ${test_predictions.std():,.2f}")

In [ ]:
# Save model
torch.save({
    'model_state_dict': best_state,
    'config': CONFIG,
    'feature_cols': feature_cols,
    'results': results_df.to_dict()
}, '../outputs/hybrid_model.pth')

print("✅ Model saved to ../outputs/hybrid_model.pth")

---
## 📝 Summary

### Model Performance Comparison

| Model | RMSE | MAE | R² | MAPE |
|-------|------|-----|-------|------|
| XGBoost (Tabular Only) | $ | $ | | % |
| Hybrid (Tabular + Satellite) | $ | $ | | % |

### Key Findings

1. **Visual Features Impact**: [Document impact of satellite images]
2. **Grad-CAM Insights**: [What areas influence predictions - green cover, roads, etc.]
3. **Improvement from Baseline**: [% improvement from tabular-only to hybrid]